In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

data = pd.read_csv('crop.csv')

print(data.head())

print(data.isnull().sum())

data = data.dropna()

encoder = LabelEncoder()
data['Crop'] = encoder.fit_transform(data['Crop'])

X = data[['Nitrogen', 'Phosphorus', 'Potassium', 'Temperature', 'Humidity', 'pH_Value', 'Rainfall']]
y = data['Crop']

X = (X - X.mean()) / X.std()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

learning_rate = 0.01
nb_epochs = 100
batch_size = 32

nb_input = X_train.shape[1]
nb_hidden1 = 256
nb_hidden2 = 256
nb_classes = len(np.unique(y))

X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = tf.keras.utils.to_categorical(y_train, nb_classes)
y_test = tf.keras.utils.to_categorical(y_test, nb_classes)

w = {
    '1': tf.Variable(tf.random.normal([nb_input, nb_hidden1], dtype=tf.float64)),
    '2': tf.Variable(tf.random.normal([nb_hidden1, nb_hidden2], dtype=tf.float64)),
    'out': tf.Variable(tf.random.normal([nb_hidden2, nb_classes], dtype=tf.float64))
}

b = {
    '1': tf.Variable(tf.random.normal([nb_hidden1], dtype=tf.float64)),
    '2': tf.Variable(tf.random.normal([nb_hidden2], dtype=tf.float64)),
    'out': tf.Variable(tf.random.normal([nb_classes], dtype=tf.float64))
}

f = {
    '1': tf.nn.relu,
    '2': tf.nn.relu,
    'out': tf.nn.softmax
}

def runNN(x):
    z1 = tf.add(tf.matmul(x, w['1']), b['1'])
    a1 = f['1'](z1)
    z2 = tf.add(tf.matmul(a1, w['2']), b['2'])
    a2 = f['2'](z2)
    z_out = tf.add(tf.matmul(a2, w['out']), b['out'])
    out = f['out'](z_out)

    pred = tf.argmax(out, 1)
    return pred, z_out

opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)

for epoch in range(nb_epochs):
    epoch_loss = 0
    nb_batches = int(len(X_train) / batch_size)
    for i in range(nb_batches):
        x_batch = X_train[i*batch_size : (i+1)*batch_size, :]
        y_batch = y_train[i*batch_size : (i+1)*batch_size]

        with tf.GradientTape() as tape:
            _, z_out = runNN(x_batch)
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=z_out, labels=y_batch))

        gradients = tape.gradient(loss, [w['1'], w['2'], w['out'], b['1'], b['2'], b['out']])
        opt.apply_gradients(zip(gradients, [w['1'], w['2'], w['out'], b['1'], b['2'], b['out']]))

        epoch_loss += loss

    epoch_loss /= nb_batches
    print(f'Epoch: {epoch+1}/{nb_epochs}, Avg loss: {epoch_loss:.5f}')

pred, _ = runNN(X_test)
pred_correct = tf.equal(pred, tf.argmax(y_test, 1))
accuracy = tf.reduce_mean(tf.cast(pred_correct, tf.float32))

print(f'Test accuracy: {accuracy:.3f}')

   Nitrogen  Phosphorus  Potassium  Temperature   Humidity  pH_Value  \
0        90          42         43    20.879744  82.002744  6.502985   
1        85          58         41    21.770462  80.319644  7.038096   
2        60          55         44    23.004459  82.320763  7.840207   
3        74          35         40    26.491096  80.158363  6.980401   
4        78          42         42    20.130175  81.604873  7.628473   

     Rainfall  Crop  
0  202.935536  Rice  
1  226.655537  Rice  
2  263.964248  Rice  
3  242.864034  Rice  
4  262.717340  Rice  
Nitrogen       0
Phosphorus     0
Potassium      0
Temperature    0
Humidity       0
pH_Value       0
Rainfall       0
Crop           0
dtype: int64
Epoch: 1/100, Avg loss: 56.29970
Epoch: 2/100, Avg loss: 4.97440
Epoch: 3/100, Avg loss: 3.60748
Epoch: 4/100, Avg loss: 4.01247
Epoch: 5/100, Avg loss: 2.69790
Epoch: 6/100, Avg loss: 1.66037
Epoch: 7/100, Avg loss: 1.33280
Epoch: 8/100, Avg loss: 1.52353
Epoch: 9/100, Avg loss: 2.345